<a href="https://colab.research.google.com/github/Bombbom/ML4SecurityProject/blob/main/dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Thu thập dataset


In [ ]:
!pip install datasets
!pip install hexbytes

In [ ]:
import re
import pandas as pd
import seaborn as sns
from hexbytes import HexBytes
import matplotlib.pyplot as plt
from datasets import load_dataset

In [ ]:
train_set = load_dataset("mwritescode/slither-audited-smart-contracts", 'big-multilabel', split='train', ignore_verifications=True)
# test_set = load_dataset("mwritescode/slither-audited-smart-contracts", 'big-multilabel', split='test', ignore_verifications=True)
# val_set = load_dataset("mwritescode/slither-audited-smart-contracts", 'big-multilabel', split='validation', ignore_verifications=True)

## Dataset info

In [ ]:
train_set.shape

In [ ]:
data = pd.DataFrame(train_set)

- 0:'access-control', 1:'arithmetic', 2:'other', 3:'reentrancy', 4:'safe', 5:'unchecked-calls'

In [ ]:
data

In [ ]:
def remove_comments(string):
    pattern = r"(\".*?\"|\'.*?\')|(/\*.*?\*/|//[^\r\n]*$)"
    # first group captures quoted strings (double or single)
    # second group captures comments (//single-line or /* multi-line */)
    regex = re.compile(pattern, re.MULTILINE|re.DOTALL)
    def _replacer(match):
        # if the 2nd group is not None, then we have captured a real comment string.
        if match.group(2) is not None:
            return "" 
        else: # otherwise, we will return the 1st group
            return match.group(1) 
    return regex.sub(_replacer, string)

In [ ]:
column = ["address", "source_code","bytecode",	"slither","access-control", "arithmetic", "other", "reentrancy", "safe", "unchecked-calls"]
df = pd.DataFrame(columns = column)

In [ ]:
for i in range(len(data)):
  row = list(data.loc[i])
  # print(list(row))
  label = row[3]
  vul = [0,0,0,0,0,0]
  for i in label:
    vul[i]=1
  row.extend(vul)
  length = len(df)
  df.loc[length] = row

In [ ]:
df.to_csv('/content/drive/MyDrive/DACN/slither-audited-smart-contracts/train_set_big_multilabel_exploid.csv',index=False)

In [ ]:
df

## Create dataset.txt

In [ ]:
count = 0
f_w = open("/content/drive/MyDrive/DACN/dataset.txt", "a")

for i in range(len(data)):
    row = list(data.loc[i])
    name = row[0] + ".sol"
    label = row[6] #reentrancy 
    count +=1
    f_w.write(str(count)+ " " + str(name) + "\n")
    f_w.write(str(name)+"\n")
    codes = []
    f = open("/content/drive/MyDrive/DACN/source_code/" + row[0] + ".sol", "r")
    lines =f.readlines()
    for line in lines:
        text = line.strip()
        if text!= "":
            codes.append(text)
    
    for k in range(0,len(codes)):
        f_w.write(codes[k] + "\n")
    
    f_w.write(str(label)+"\n")
    f_w.write("---------------------------------" + "\n")

## Dataset (code snippet)